#### The Business Problem and Dataset ####

As a junior data scientist at TeleConnect, a telecommunications company, you've been tasked with developing a model to predict which customers are likely to cancel their service (churn) in the next month. Your manager believes that identifying at-risk customers early could allow the retention team to take proactive measures and reduce overall churn rates.

The CEO has highlighted that acquiring a new customer costs five times more than retaining an existing one, making this project a high priority. You decide to implement a k-NN model because:

- It's intuitive to explain to stakeholders.
- The company has labeled historical data on which customers churned.
- You need to predict a binary outcome (churn or no churn).
- The transparent nature of k-NN allows the retention team to see which existing customers are most similar to potential churners.

##### Dataset Details #####

You're provided with a dataset containing information on 5,000 customers from the past year, with the following features:

- Monthly bill amount
- Contract length (in months)
- Total service calls in the last 3 months
- Service issues reported in the last 3 months
- Total products/services subscribed
- Months as a customer
- Average monthly usage (in GB)
- Churn status (Yes/No)

In [ ]:
# Load tools

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Prepare the data

# Load the dataset
telecom_data = pd.read_csv('customer_data.csv')

# Split features and target
X = telecom_data.drop('churn_status', axis=1)
y = telecom_data['churn_status']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Finding the Optimal K Value #####

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

# Testing different k values
k_range = range(1, 31)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    k_scores.append(scores.mean())

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(k_range, k_scores)
plt.xlabel('Value of K')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Accuracy for Different Values of K')
plt.grid(True)
plt.show()

# Find the optimal k
optimal_k = k_range[k_scores.index(max(k_scores))]
print(f"The optimal value of k is: {optimal_k}")

##### Building the k-NN Model #####

In [ ]:
# Creating the model with the optimal k
knn_model = KNeighborsClassifier(n_neighbors=optimal_k)

# Training the model
knn_model.fit(X_train_scaled, y_train)

# Making predictions
y_pred = knn_model.predict(X_test_scaled)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

##### Implementing Distance-Weighting Vote #####

In [ ]:
# Creating a weighted k-NN model
weighted_knn = KNeighborsClassifier(
    n_neighbors=optimal_k,
    weights='distance'  # Use distance-weighted voting
)

# Training the model
weighted_knn.fit(X_train_scaled, y_train)

# Making predictions
y_pred_weighted = weighted_knn.predict(X_test_scaled)

# Evaluating the weighted model
w_accuracy = accuracy_score(y_test, y_pred_weighted)
w_conf_matrix = confusion_matrix(y_test, y_pred_weighted)
w_class_report = classification_report(y_test, y_pred_weighted)

print(f"Weighted Model Accuracy: {w_accuracy:.4f}")
print("\nWeighted Confusion Matrix:")
print(w_conf_matrix)
print("\nWeighted Classification Report:")
print(w_class_report)